In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
from copy import copy

from tooltoad.vis import MolTo3DGrid
from tooltoad.chemutils import ac2mol

from rdkit import Chem
from rdkit.Chem import rdDetermineBonds, rdMolAlign
from rdkit.Chem.AllChem import ETKDGv3, EmbedMolecule
from rdkit.Chem.rdchem import RWMol

from frust.utils.mols import fix_cat_frag, get_molecule_name
from frust.embedder import embed_ts

In [35]:
def transformer_ts3(
    ligand_smiles="C1=CC=CO1",
    ts_guess_struct="../structures/ts3.xyz",
    bonds_to_remove = [(10, 11), (10,20)],
    pre_name="TS3",
    embed_ready=True,
):
    # --- Read TS Guess Structure --- #
    try:
        with open(ts_guess_struct, 'r') as file:
            xyz_block = file.read()
    except FileNotFoundError:
        print(f"Error: Transition state structure file not found: {ts_guess_struct}")
        raise
    except PermissionError:
        print(f"Error: Permission denied when accessing file: {ts_guess_struct}")
        raise
    except IOError as e:
        print(f"Error: Failed to read transition state structure file {ts_guess_struct}: {e}")
        raise
    except Exception as e:
        print(f"Unexpected error loading transition state structure from {ts_guess_struct}: {e}")
        raise

    # --- Determine Connectivity --- #
    from rdkit import Chem
    ts = Chem.MolFromXYZBlock(xyz_block)
    rdDetermineBonds.DetermineConnectivity(ts, useVdw=True)
    ts_rw = RWMol(ts)

    bonds_to_remove = bonds_to_remove
    for bond in bonds_to_remove:
        ts_rw.RemoveBond(bond[0], bond[1])
    ts_rw_origin = copy(ts_rw)

    # --- Find ligand in guess ts structure --- #
    ts_ligand_pattern = Chem.MolFromSmarts("S1CCCC1")
    old_ring_match    = ts_rw.GetSubstructMatch(ts_ligand_pattern)  # e.g. (5,6,7,8,9)

    # --- Find unique positions and check that they are valid cH --- #
    lig_mol = Chem.MolFromSmiles(ligand_smiles)
    lig_mol = Chem.AddHs(lig_mol)

    cH_patt = Chem.MolFromSmarts('[cH]')
    matches = lig_mol.GetSubstructMatches(cH_patt)
    cH_atoms = [ind[0] for ind in matches]

    atom_rank = list(Chem.CanonicalRankAtoms(lig_mol, breakTies=False))

    def find_unique_atoms(lst):
        seen = set()
        result = []
        for i, x in enumerate(lst):
            if x not in seen:
                result.append(i)
                seen.add(x)
        return result

    unique_atoms = find_unique_atoms(atom_rank)
    unique_cH = set(unique_atoms).intersection(set(cH_atoms))
    unique_cH = tuple(unique_cH)

    # --- Create aligned maps --- #
    old_active_site = old_ring_match[0:3]

    maps = []
    for a in unique_cH:
        C_pos = lig_mol.GetAtomWithIdx(a)
        nbs = []
        for nb in C_pos.GetNeighbors():
            if nb.GetAtomicNum() == 1:
                pass # hydrogen
            else:
                nbs.append(nb.GetIdx())
        
        nbs.insert(1, C_pos.GetIdx())
        
        map = []
        for nb, aa in zip(nbs, old_active_site):
            map.append((nb, aa))
        maps.append(map)

    # --- Loop through each map a.k.a reactive position and create the molecule --- #
    params = ETKDGv3()
    params.randomSeed = 0xF00D  # Use any integer seed
    ts_mols = {}

    lig_mol_original = copy(lig_mol)

    for map in maps:
        lig_mol = lig_mol_original
        rpos = map[1][0]
        EmbedMolecule(lig_mol, params)
        ts_rw = Chem.RWMol(ts_rw_origin)
        rdMolAlign.AlignMol(lig_mol, ts_rw, atomMap=map)

        # --- remove hydrogen from the reacting carbon --- #
        chosen_carbon_idx = rpos
        chosen_carbon = lig_mol.GetAtomWithIdx(chosen_carbon_idx)

        for nb in chosen_carbon.GetNeighbors():
            if nb.GetAtomicNum() == 1:  # hydrogen
                lig_mol_rw = RWMol(lig_mol)
                lig_mol_rw.RemoveAtom(nb.GetIdx())
                lig_mol = lig_mol_rw.GetMol()
                break

        # --- Remove old ligand and determine bond order (to get aromaticity correct for the catalyst) --- #
        n_pattern_full = Chem.MolFromSmiles("C1CCCS1")
        n_old_indices = ts_rw.GetSubstructMatch(n_pattern_full)

        atoms_to_remove = set()
        for idx in n_old_indices:
            atom = ts_rw.GetAtomWithIdx(idx)
            atoms_to_remove.add(idx)
            for neighbor in atom.GetNeighbors():
                if neighbor.GetAtomicNum() == 1:  # Check if hydrogen
                    atoms_to_remove.add(neighbor.GetIdx())

        for idx in sorted(atoms_to_remove, reverse=True):
            ts_rw.RemoveAtom(idx)

        frags = Chem.GetMolFrags(ts_rw, asMols=True)

        frag0 = fix_cat_frag(frags[0])
        rdDetermineBonds.DetermineBonds(frags[1])

        ts_rw = RWMol(Chem.CombineMols(frag0, frags[1]))

        # --- Combine ligand and catalyst, add temporary bonds, and set temporary formal charges ---
        ts_combined = Chem.CombineMols(ts_rw, lig_mol)
        ts_rw_combined = RWMol(ts_combined)

        offset = ts_rw.GetNumAtoms()
        reactive_C = rpos + offset

        cat_pat = Chem.MolFromSmarts('[B]-c1ccccc1-[N]')
        B_cat_idx = ts_rw_combined.GetSubstructMatches(cat_pat)[0][0]
        N_cat_idx = ts_rw_combined.GetSubstructMatches(cat_pat)[0][7]
        B_nbs = ts_rw_combined.GetAtomWithIdx(B_cat_idx).GetNeighbors()
        Hs_on_B = [nb.GetIdx() for nb in B_nbs if nb.GetAtomicNum() == 1]

        ts_rw_combined.AddBond(reactive_C, B_cat_idx, Chem.BondType.SINGLE)

        pin_pat = Chem.MolFromSmarts('[B]1OC(C(O1)(C)C)(C)C')
        B_pin_idx = ts_rw_combined.GetSubstructMatches(pin_pat)[0][0]
        B_pin_nbs = ts_rw_combined.GetAtomWithIdx(B_pin_idx).GetNeighbors()
        H_pin_idx = B_pin_nbs[0].GetIdx()

        atom_indices_to_keep = [B_cat_idx, N_cat_idx]
        atom_indices_to_keep.extend(Hs_on_B)
        atom_indices_to_keep.extend([B_pin_idx])
        atom_indices_to_keep.extend([H_pin_idx, reactive_C])

        if embed_ready:
            pass
            ts_rw_combined.AddBond(B_cat_idx, B_pin_idx, Chem.BondType.SINGLE)
            b_pin_atom = ts_rw_combined.GetAtomWithIdx(B_pin_idx)
            b_cat_atom = ts_rw_combined.GetAtomWithIdx(B_cat_idx)
            b_pin_atom.SetFormalCharge(2)
            b_cat_atom.SetFormalCharge(2)
            
            # c_atom = ts_rw_combined.GetAtomWithIdx(reactive_C)
            # c_atom.SetFormalCharge(0)

        mol_name = get_molecule_name(ligand_smiles)
        ts_mols[f'{pre_name}({mol_name}_rpos({rpos}))'] = (ts_rw_combined, atom_indices_to_keep, ligand_smiles)

    return ts_mols

ts_dict = transformer_ts3(embed_ready=True)

In [36]:
ts_dict

{'TS3(furan_rpos(0))': (<rdkit.Chem.rdchem.RWMol at 0x16c006890>,
  [10, 12, 11, 22, 21, 43],
  'C1=CC=CO1'),
 'TS3(furan_rpos(1))': (<rdkit.Chem.rdchem.RWMol at 0x16c010ea0>,
  [10, 12, 11, 22, 21, 44],
  'C1=CC=CO1')}

In [37]:
MolTo3DGrid(list(ts_dict.values())[0][0], kekulize=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [38]:
embeds = embed_ts(ts_dict, ts_type="TS3-NEW", n_confs=2, optimize=True)

[10, 12, 11, 22, 21, 43]
Embedded 2 conformers on atom 43
[10, 12, 11, 22, 21, 44]
Embedded 2 conformers on atom 44


In [39]:
MolTo3DGrid(list(embeds.values())[0][0])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [40]:
from frust.stepper import Stepper

step = Stepper(["C1=CC=CO1"], step_type="TS3-NEW", save_output_dir=False)
df0 = step.build_initial_df(embeds)
df0

2025-07-17 10:04:41 INFO  frust.stepper: Working dir: .


,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff
0,TS3(furan_rpos(0)),furan,0,"[10, 12, 11, 22, 21, 43]",0,C1=CC=CO1,"[C, C, C, C, C, C, H, H, H, H, B, H, N, C, H, ...","[(-2.284245481738126, 1.6198931268243966, -2.5...",7809.246713
1,TS3(furan_rpos(0)),furan,0,"[10, 12, 11, 22, 21, 43]",1,C1=CC=CO1,"[C, C, C, C, C, C, H, H, H, H, B, H, N, C, H, ...","[(-4.195088897994577, -2.2886381855054294, 0.5...",10666.950678
2,TS3(furan_rpos(1)),furan,1,"[10, 12, 11, 22, 21, 44]",0,C1=CC=CO1,"[C, C, C, C, C, C, H, H, H, H, B, H, N, C, H, ...","[(-3.875939151962986, -2.269315583375933, -2.0...",8211.988725
3,TS3(furan_rpos(1)),furan,1,"[10, 12, 11, 22, 21, 44]",1,C1=CC=CO1,"[C, C, C, C, C, C, H, H, H, H, B, H, N, C, H, ...","[(-4.326001365259978, -2.4762392336182413, -0....",9361.341918


In [48]:
df1 = step.xtb(df0, options={"gfnff": None, "opt": None}, constraint=True)
df1

2025-07-17 10:12:59 INFO  frust.stepper: [xtb-gfnff-opt] row 0 (TS3(furan_rpos(0)))…
2025-07-17 10:12:59 INFO  frust.stepper: [xtb-gfnff-opt] row 1 (TS3(furan_rpos(0)))…
2025-07-17 10:12:59 INFO  frust.stepper: [xtb-gfnff-opt] row 2 (TS3(furan_rpos(1)))…
2025-07-17 10:12:59 INFO  frust.stepper: [xtb-gfnff-opt] row 3 (TS3(furan_rpos(1)))…


,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff,xtb-gfnff-opt-electronic_energy,xtb-gfnff-opt-normal_termination,xtb-gfnff-opt-opt_coords
0,TS3(furan_rpos(0)),furan,0,"[10, 12, 11, 22, 21, 43]",0,C1=CC=CO1,"[C, C, C, C, C, C, H, H, H, H, B, H, N, C, H, ...","[(-2.284245481738126, 1.6198931268243966, -2.5...",7809.246713,-8.480827,True,"[[-2.49436120393025, 0.54352475209926, -2.9873..."
1,TS3(furan_rpos(0)),furan,0,"[10, 12, 11, 22, 21, 43]",1,C1=CC=CO1,"[C, C, C, C, C, C, H, H, H, H, B, H, N, C, H, ...","[(-4.195088897994577, -2.2886381855054294, 0.5...",10666.950678,-8.480879,True,"[[-4.38293617264813, -2.56389874587409, 0.6373..."
2,TS3(furan_rpos(1)),furan,1,"[10, 12, 11, 22, 21, 44]",0,C1=CC=CO1,"[C, C, C, C, C, C, H, H, H, H, B, H, N, C, H, ...","[(-3.875939151962986, -2.269315583375933, -2.0...",8211.988725,-8.453233,True,"[[-3.94646040856551, -2.41063449839222, -2.122..."
3,TS3(furan_rpos(1)),furan,1,"[10, 12, 11, 22, 21, 44]",1,C1=CC=CO1,"[C, C, C, C, C, C, H, H, H, H, B, H, N, C, H, ...","[(-4.326001365259978, -2.4762392336182413, -0....",9361.341918,-8.457203,True,"[[-4.53399737303861, -2.45972154248935, 0.0523..."


In [50]:
idx = 0
atoms = df1["atoms"].iloc[idx]
coords1 = df1["coords_embedded"].iloc[idx]
coords2 = df1["xtb-gfnff-opt-opt_coords"].iloc[idx]
all_coords = [coords1, coords2]
all_mols = [ac2mol(atoms, c) for c in all_coords]
MolTo3DGrid(all_mols, legends=['embed', 'xtb-opt'])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [43]:
from tooltoad.chemutils import xyz2mol

with open("../structures/ts3.xyz") as file:
    xyzblock = file.read()

mol = RWMol(xyz2mol(xyzblock))
mol.RemoveBond(10, 20)

MolTo3DGrid(mol)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.